In [40]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50


In [41]:
mnist = tf.keras.datasets.mnist  
(x_train, y_train),(x_test, y_test) = mnist.load_data()  


In [42]:
#download  resnet weights < resorted to download weights used in imagenet>
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")
        
# download(r'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels.h5','resnet_weight_top.h5')
# download(r'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels_notop.h5','resnet_weight.h5')
print("All the files are downloaded")

All the files are downloaded


In [43]:
number_of_classes = 10
validation_percentage = 0.2
resnet_path = "resnet_weight_top.h5"

In [44]:
def prepare_data_for_resnet50(data_to_transform):
    data = data_to_transform.copy()
    data = data.reshape(-1, 28, 28) / 255
    data = X_rgb = np.stack([data, data, data], axis=-1)
    return data

print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [45]:
x_train  = prepare_data_for_resnet50(x_train)
x_test = prepare_data_for_resnet50(x_test)
y_train = to_categorical(y_train,number_of_classes)
y_test = to_categorical(y_test,number_of_classes)

In [46]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28, 3)
(10000, 28, 28, 3)
(60000, 10)
(10000, 10)


In [47]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg',weights='imagenet'))
model.summary()
#if resnet layers are made untrainable , highest accuracy that can be achieved is 68% train and  78% test

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [48]:
model.add(Flatten())
model.add(Dropout(0.50))
model.add(Dense(number_of_classes, activation='softmax'))

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [50]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
epochs = 3
batch_size = 64

In [52]:
model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test , y_test) , batch_size=batch_size)

Epoch 1/3
938/938 [==============================] - 111s 118ms/step - loss: 0.2837 - accuracy: 0.9426 - val_loss: 1.8120 - val_accuracy: 0.7502
Epoch 2/3
938/938 [==============================] - 109s 116ms/step - loss: 0.1641 - accuracy: 0.9636 - val_loss: 0.0597 - val_accuracy: 0.9819
Epoch 3/3
938/938 [==============================] - 109s 116ms/step - loss: 0.0827 - accuracy: 0.9806 - val_loss: 0.0566 - val_accuracy: 0.9848
